In [4]:
import pickle
import re
import sentencepiece as spm

%cd ../../data/nagoya_corpus/

C:\Users\yuuki\PycharmProjects\NLP\data\nagoya_corpus


## データ読み込み

In [5]:
data = []
for i in range(1, 130):
    with open('data{}.txt'.format(str(i).zfill(3)), encoding='utf-8') as f:
        data.extend(f.readlines())

## データ整形

In [6]:
person_pat = re.compile(r'[A-Z][0-9]+[：]')
unk_pat = re.compile(r'＊＊＊')
koyu_pat = re.compile(r'[＊].+[＊]')
kakko_pat = re.compile(r'[（][^（）]+[）]')
tagu_pat = re.compile(r'[＜][^＜＞]+[＞]')

data = [d.rstrip() for d in data if d[0] != '＠']
buf = []
for d in data:
    if not person_pat.match(d):
        buf[-1] += d
    else:
        buf.append(person_pat.sub('', d))
data = buf

# ＊＊＊を<unk>に置き換え
buf = []
for d in data:
    buf.append(unk_pat.sub('[UNK]', d))
data = buf

# ＊固有名詞＊を<unk>に置き換え
buf = []
for d in data:
    buf.append(koyu_pat.sub('[UNK]', d))
data = buf

# （・・・）を削除
buf = []
for d in data:
    buf.append(kakko_pat.sub('', d))
data = buf

# ＜・・・＞を削除
buf = []
for d in data:
    buf.append(tagu_pat.sub('', d))
data = buf

In [7]:
with open('data.txt', 'w', encoding='utf-8') as f:
    for d in data:
        f.write(d+'\n')

In [8]:
spm.SentencePieceTrainer.Train('--control_symbols=<pad> --user_defined_symbols=[UNK] --input=data.txt --model_prefix=trained_model --vocab_size=8000 --add_dummy_prefix=false')

True

In [9]:
sp = spm.SentencePieceProcessor()

In [10]:
sp.Load('trained_model.model')

True

In [11]:
sp.EncodeAsIds('')

[]

In [12]:
sp.PieceToId('')

0